In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('../../datasets/Hotel Reservations.csv')
df.set_index('Booking_ID', inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36275 entries, INN00001 to INN36275
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          36275 non-null  int64  
 1   no_of_children                        36275 non-null  int64  
 2   no_of_weekend_nights                  36275 non-null  int64  
 3   no_of_week_nights                     36275 non-null  int64  
 4   type_of_meal_plan                     36275 non-null  object 
 5   required_car_parking_space            36275 non-null  int64  
 6   room_type_reserved                    36275 non-null  object 
 7   lead_time                             36275 non-null  int64  
 8   arrival_year                          36275 non-null  int64  
 9   arrival_month                         36275 non-null  int64  
 10  arrival_date                          36275 non-null  int64  
 11  market_seg

In [24]:
df['booking_status'].value_counts()

booking_status
1    24390
0    11885
Name: count, dtype: int64

In [3]:
df['type_of_meal_plan']

Booking_ID
INN00001     Meal Plan 1
INN00002    Not Selected
INN00003     Meal Plan 1
INN00004     Meal Plan 1
INN00005    Not Selected
                ...     
INN36271     Meal Plan 1
INN36272     Meal Plan 1
INN36273     Meal Plan 1
INN36274    Not Selected
INN36275     Meal Plan 1
Name: type_of_meal_plan, Length: 36275, dtype: object

In [5]:
# LabelEncoder for df['type_of_meal_plan']
from sklearn.calibration import LabelEncoder


labelencoder = LabelEncoder()

df['type_of_meal_plan'] = labelencoder.fit_transform(df['type_of_meal_plan'])
df['room_type_reserved'] = labelencoder.fit_transform(df['room_type_reserved'])
df['market_segment_type'] = labelencoder.fit_transform(df['market_segment_type'])

df['booking_status'] = labelencoder.fit_transform(df['booking_status'])

In [6]:
df['type_of_meal_plan']

Booking_ID
INN00001    0
INN00002    3
INN00003    0
INN00004    0
INN00005    3
           ..
INN36271    0
INN36272    0
INN36273    0
INN36274    3
INN36275    0
Name: type_of_meal_plan, Length: 36275, dtype: int64

In [25]:
print(df['type_of_meal_plan'].unique())
print(df['room_type_reserved'].unique())
print(df['market_segment_type'].unique())

[0 3 1 2]
[0 3 1 5 4 6 2]
[3 4 2 0 1]


In [26]:
df.describe().T.to_latex(
    './hotel_describe.tex',
    float_format="%.2f",
    bold_rows=True
)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36275 entries, INN00001 to INN36275
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          36275 non-null  int64  
 1   no_of_children                        36275 non-null  int64  
 2   no_of_weekend_nights                  36275 non-null  int64  
 3   no_of_week_nights                     36275 non-null  int64  
 4   type_of_meal_plan                     36275 non-null  int64  
 5   required_car_parking_space            36275 non-null  int64  
 6   room_type_reserved                    36275 non-null  int64  
 7   lead_time                             36275 non-null  int64  
 8   arrival_year                          36275 non-null  int64  
 9   arrival_month                         36275 non-null  int64  
 10  arrival_date                          36275 non-null  int64  
 11  market_seg

In [28]:
df.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
Booking_ID,,,,,,,,,,,,,,,,,,
INN00001,2,0,1,2,0,0,0,224,2017,10,2,3,0,0,0,65.00,0,1
INN00002,2,0,2,3,3,0,0,5,2018,11,6,4,0,0,0,106.68,1,1
INN00003,1,0,2,1,0,0,0,1,2018,2,28,4,0,0,0,60.00,0,0
INN00004,2,0,0,2,0,0,0,211,2018,5,20,4,0,0,0,100.00,0,0
INN00005,2,0,1,1,3,0,0,48,2018,4,11,4,0,0,0,94.50,0,0


In [29]:
# Scaler
scaler = RobustScaler()

features = pd.DataFrame(scaler.fit_transform(df.drop('booking_status', axis=1)), columns=df.columns[:-1])
target = df['booking_status']

In [30]:
features

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.532110,-1.0,0.4,-0.933333,-1.0,0.0,0.0,0.0,-0.867758,0.0
1,0.0,0.0,0.5,0.5,3.0,0.0,0.0,-0.477064,0.0,0.6,-0.666667,0.0,0.0,0.0,0.0,0.182116,1.0
2,-1.0,0.0,0.5,-0.5,0.0,0.0,0.0,-0.513761,0.0,-1.2,0.800000,0.0,0.0,0.0,0.0,-0.993703,0.0
3,0.0,0.0,-0.5,0.0,0.0,0.0,0.0,1.412844,0.0,-0.6,0.266667,0.0,0.0,0.0,0.0,0.013854,0.0
4,0.0,0.0,0.0,-0.5,3.0,0.0,0.0,-0.082569,0.0,-0.8,-0.333333,0.0,0.0,0.0,0.0,-0.124685,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,1.0,0.0,0.5,2.0,0.0,0.0,3.0,0.256881,0.0,0.0,-0.866667,0.0,0.0,0.0,0.0,1.721662,1.0
36271,0.0,0.0,0.0,0.5,0.0,0.0,0.0,1.568807,0.0,0.4,0.066667,0.0,0.0,0.0,0.0,-0.214106,2.0
36272,0.0,0.0,0.5,2.0,0.0,0.0,0.0,0.834862,0.0,-0.2,-1.000000,0.0,0.0,0.0,0.0,-0.026700,2.0
36273,0.0,0.0,-0.5,0.5,3.0,0.0,0.0,0.055046,0.0,-0.8,0.333333,0.0,0.0,0.0,0.0,-0.124685,0.0


In [31]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=.2, random_state=2023)

In [32]:
# # Hiperparameters with GridSearchCV for Models: GaussianNB, Decision Tree, KNN and Random Forest
# from sklearn.model_selection import GridSearchCV


# # GaussianNB
# params_gnb = {'var_smoothing': np.logspace(0, -9, num=100)}
# grid_gnb = GridSearchCV(estimator=GaussianNB(), param_grid=params_gnb, cv=5, verbose=1, n_jobs=-1)
# grid_gnb.fit(X_train, y_train)
# print(f"Model: GaussianNB | Best params: {grid_gnb.best_params_}\n")

# # Decision Tree
# params_dt = {'criterion': ['gini', 'entropy'],
#                 'max_depth': [3, 5, 7, 9, 11, 13, 15],
#                 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9],
#                 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9]}
# grid_dt = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=params_dt, cv=5, verbose=1, n_jobs=-1)
# grid_dt.fit(X_train, y_train)
# print(f"Model: DT | Best params: {grid_dt.best_params_}\n")

# # KNN
# params_knn = {'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
#                 'weights': ['uniform', 'distance'],
#                 'metric': ['euclidean', 'manhattan', 'minkowski']}
# grid_knn = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=params_knn, cv=5, verbose=1, n_jobs=-1)
# grid_knn.fit(X_train, y_train)
# print(f"Model: KNN | Best params: {grid_knn.best_params_}\n")

# # Random Forest
# params_rf = {'n_estimators': [100, 200, 300, 400, 500],
#              'criterion': ['gini', 'entropy'],
#                 'max_depth': [3, 5, 7, 9, 11, 13, 15],
#                 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9],
#                 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9]}
# grid_rf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=params_rf, cv=5, verbose=1, n_jobs=-1)
# grid_rf.fit(X_train, y_train)
# print(f"Model: RF | Best params: {grid_rf.best_params_}\n")

In [33]:
models_with_best_params = {
    'GaussianNB': GaussianNB(var_smoothing=0.8111308307896871),
    'DT': DecisionTreeClassifier(criterion='gini', max_depth=15, min_samples_leaf=1, min_samples_split=2),
    'KNN': KNeighborsClassifier(metric='manhattan', n_neighbors=15, weights='distance'),
    'RF': RandomForestClassifier(criterion='gini', max_depth=15, min_samples_leaf=1, min_samples_split=3, n_estimators=200),
}

In [34]:
# Metrics: Accuracy, Precision, Recall, F1
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def get_metrics():
    metrics = {
        'accuracy': accuracy_score,
        'precision': precision_score,
        'recall': recall_score,
        'f1': f1_score,
        'roc': roc_auc_score
    }
    return metrics


def get_metrics_df(y_true, y_pred, name):
    metrics = get_metrics()
    df = pd.DataFrame()
    for metric_name, metric in metrics.items():
        df[metric_name] = [metric(y_true, y_pred)]
    return df

In [35]:
SEEDS = [24, 42, 206, 602, 412, 214, 754, 457, 2023, 3202]

In [36]:
run_counter = 0
results = dict()

# Splitting dataset
X = df.drop(columns='booking_status')
y = df['booking_status']

for seed in SEEDS:
    run_counter += 1
    print(f'%{run_counter} - Running for seed: {seed}')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=seed)

    models_with_best_params = {
        'GaussianNB': GaussianNB(var_smoothing=0.8111308307896871),
        'DT': DecisionTreeClassifier(criterion='gini', max_depth=15, min_samples_leaf=1, min_samples_split=2),
        'KNN': KNeighborsClassifier(metric='manhattan', n_neighbors=15, weights='distance'),
        'RF': RandomForestClassifier(criterion='gini', max_depth=15, min_samples_leaf=1, min_samples_split=3, n_estimators=200),
    }

    for name, model in models_with_best_params.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        results[f"{model.__class__.__name__}_{seed}"] = get_metrics_df(y_test, y_pred, name)

%1 - Running for seed: 24
%2 - Running for seed: 42
%3 - Running for seed: 206
%4 - Running for seed: 602
%5 - Running for seed: 412
%6 - Running for seed: 214
%7 - Running for seed: 754
%8 - Running for seed: 457
%9 - Running for seed: 2023
%10 - Running for seed: 3202


In [37]:
results.keys()

dict_keys(['GaussianNB_24', 'DecisionTreeClassifier_24', 'KNeighborsClassifier_24', 'RandomForestClassifier_24', 'GaussianNB_42', 'DecisionTreeClassifier_42', 'KNeighborsClassifier_42', 'RandomForestClassifier_42', 'GaussianNB_206', 'DecisionTreeClassifier_206', 'KNeighborsClassifier_206', 'RandomForestClassifier_206', 'GaussianNB_602', 'DecisionTreeClassifier_602', 'KNeighborsClassifier_602', 'RandomForestClassifier_602', 'GaussianNB_412', 'DecisionTreeClassifier_412', 'KNeighborsClassifier_412', 'RandomForestClassifier_412', 'GaussianNB_214', 'DecisionTreeClassifier_214', 'KNeighborsClassifier_214', 'RandomForestClassifier_214', 'GaussianNB_754', 'DecisionTreeClassifier_754', 'KNeighborsClassifier_754', 'RandomForestClassifier_754', 'GaussianNB_457', 'DecisionTreeClassifier_457', 'KNeighborsClassifier_457', 'RandomForestClassifier_457', 'GaussianNB_2023', 'DecisionTreeClassifier_2023', 'KNeighborsClassifier_2023', 'RandomForestClassifier_2023', 'GaussianNB_3202', 'DecisionTreeClassif

In [38]:
gaussian_keys = [key for key in results.keys() if 'Gaussian' in key]
dt_keys = [key for key in results.keys() if 'DecisionTreeClassifier' in key]
knn_keys = [key for key in results.keys() if 'KNeighborsClassifier' in key]
rf_keys = [key for key in results.keys() if 'RandomForestClassifier' in key]

In [39]:
print(len(gaussian_keys))
assert len(gaussian_keys) == len(dt_keys) == len(knn_keys) == len(rf_keys)

10


In [51]:
from tabulate import tabulate


# Gaussian accuracy mean
gaussian_accuracy_mean = np.mean([results[key]['accuracy'].values[0] for key in gaussian_keys])

# Gaussian accuracy std
gaussian_accuracy_std = np.std([results[key]['accuracy'].values[0] for key in gaussian_keys])

# Gaussian precision mean
gaussian_precision_mean = np.mean([results[key]['precision'].values[0] for key in gaussian_keys])

# Gaussian precision std
gaussian_precision_std = np.std([results[key]['precision'].values[0] for key in gaussian_keys])

# Gaussian recall mean
gaussian_recall_mean = np.mean([results[key]['recall'].values[0] for key in gaussian_keys])

# Gaussian recall std
gaussian_recall_std = np.std([results[key]['recall'].values[0] for key in gaussian_keys])

# Gaussian f1 mean
gaussian_f1_mean = np.mean([results[key]['f1'].values[0] for key in gaussian_keys])

# Gaussian f1 std
gaussian_f1_std = np.std([results[key]['f1'].values[0] for key in gaussian_keys])

# Gaussian roc mean
gaussian_roc_mean = np.mean([results[key]['roc'].values[0] for key in gaussian_keys])

# Gaussian roc std
gaussian_roc_std = np.std([results[key]['roc'].values[0] for key in gaussian_keys])


# Print tabulate
print(tabulate(
    [
        ['GaussianNB', 
         gaussian_accuracy_mean, gaussian_accuracy_std, 
         gaussian_precision_mean, gaussian_precision_std, 
         gaussian_recall_mean, gaussian_recall_std, 
         gaussian_f1_mean, gaussian_f1_std, 
         gaussian_roc_mean, gaussian_roc_std],
    ],
    headers=['Model', 'Accuracy Mean', 'Accuracy Std', 
             'Precision Mean', 'Precision Std', 
             'Recall Mean', 'Recall Std', 
             'F1 Mean', 'F1 Std', 
             'ROC Mean', 'ROC Std'],
    tablefmt='psql',
    floatfmt=(".4"),
))


# DT accuracy mean
dt_accuracy_mean = np.mean([results[key]['accuracy'].values[0] for key in dt_keys])

# DT accuracy std
dt_accuracy_std = np.std([results[key]['accuracy'].values[0] for key in dt_keys])

# DT precision mean
dt_precision_mean = np.mean([results[key]['precision'].values[0] for key in dt_keys])

# DT precision std
dt_precision_std = np.std([results[key]['precision'].values[0] for key in dt_keys])

# DT recall mean
dt_recall_mean = np.mean([results[key]['recall'].values[0] for key in dt_keys])

# DT recall std
dt_recall_std = np.std([results[key]['recall'].values[0] for key in dt_keys])

# DT f1 mean
dt_f1_mean = np.mean([results[key]['f1'].values[0] for key in dt_keys])

# DT f1 std
dt_f1_std = np.std([results[key]['f1'].values[0] for key in dt_keys])

# DT roc mean
dt_roc_mean = np.mean([results[key]['roc'].values[0] for key in dt_keys])

# DT roc std
dt_roc_std = np.std([results[key]['roc'].values[0] for key in dt_keys])


# Print tabulate
print(tabulate(
    [
        ['DecisionTreeClassifier', dt_accuracy_mean, dt_accuracy_std, 
         dt_precision_mean, dt_precision_std, 
         dt_recall_mean, dt_recall_std, 
         dt_f1_mean, dt_f1_std, 
         dt_roc_mean, dt_roc_std],
    ],
    headers=['Model', 'Accuracy Mean', 'Accuracy Std', 
             'Precision Mean', 'Precision Std', 
             'Recall Mean', 'Recall Std', 
             'F1 Mean', 'F1 Std', 
             'ROC Mean', 'ROC Std'],
    tablefmt='psql',
    floatfmt=(".4"),
))


# KNN accuracy mean
knn_accuracy_mean = np.mean([results[key]['accuracy'].values[0] for key in knn_keys])

# KNN accuracy std
knn_accuracy_std = np.std([results[key]['accuracy'].values[0] for key in knn_keys])

# KNN precision mean
knn_precision_mean = np.mean([results[key]['precision'].values[0] for key in knn_keys])

# KNN precision std
knn_precision_std = np.std([results[key]['precision'].values[0] for key in knn_keys])

# KNN recall mean
knn_recall_mean = np.mean([results[key]['recall'].values[0] for key in knn_keys])

# KNN recall std
knn_recall_std = np.std([results[key]['recall'].values[0] for key in knn_keys])

# KNN f1 mean
knn_f1_mean = np.mean([results[key]['f1'].values[0] for key in knn_keys])

# KNN f1 std
knn_f1_std = np.std([results[key]['f1'].values[0] for key in knn_keys])

# KNN roc mean
knn_roc_mean = np.mean([results[key]['roc'].values[0] for key in knn_keys])

# KNN roc std
knn_roc_std = np.std([results[key]['roc'].values[0] for key in knn_keys])


# Print tabulate
print(tabulate(
    [
        ['KNeighborsClassifier', knn_accuracy_mean, knn_accuracy_std, 
         knn_precision_mean, knn_precision_std, 
         knn_recall_mean, knn_recall_std, 
         knn_f1_mean, knn_f1_std, 
         knn_roc_mean, knn_roc_std],
    ],
    headers=['Model', 'Accuracy Mean', 'Accuracy Std', 
             'Precision Mean', 'Precision Std', 
             'Recall Mean', 'Recall Std', 
             'F1 Mean', 'F1 Std', 
             'ROC Mean', 'ROC Std'],
    tablefmt='psql',
    floatfmt=(".4"),
))


# RF accuracy mean
rf_accuracy_mean = np.mean([results[key]['accuracy'].values[0] for key in rf_keys])

# RF accuracy std
rf_accuracy_std = np.std([results[key]['accuracy'].values[0] for key in rf_keys])

# RF precision mean
rf_precision_mean = np.mean([results[key]['precision'].values[0] for key in rf_keys])

# RF precision std
rf_precision_std = np.std([results[key]['precision'].values[0] for key in rf_keys])

# RF recall mean
rf_recall_mean = np.mean([results[key]['recall'].values[0] for key in rf_keys])

# RF recall std
rf_recall_std = np.std([results[key]['recall'].values[0] for key in rf_keys])

# RF f1 mean
rf_f1_mean = np.mean([results[key]['f1'].values[0] for key in rf_keys])

# RF f1 std
rf_f1_std = np.std([results[key]['f1'].values[0] for key in rf_keys])

# RF roc mean
rf_roc_mean = np.mean([results[key]['roc'].values[0] for key in rf_keys])

# RF roc std
rf_roc_std = np.std([results[key]['roc'].values[0] for key in rf_keys])


# Print tabulate
print(tabulate(
    [
        ['RandomForestClassifier', rf_accuracy_mean, rf_accuracy_std, 
         rf_precision_mean, rf_precision_std, 
         rf_recall_mean, rf_recall_std, 
         rf_f1_mean, rf_f1_std, 
         rf_roc_mean, rf_roc_std],
    ],
    headers=['Model', 'Accuracy Mean', 'Accuracy Std', 
             'Precision Mean', 'Precision Std', 
             'Recall Mean', 'Recall Std', 
             'F1 Mean', 'F1 Std', 
             'ROC Mean', 'ROC Std'],
    tablefmt='psql',
    floatfmt=(".4"),
))

+------------+-----------------+----------------+------------------+-----------------+---------------+--------------+-----------+----------+------------+-----------+
| Model      |   Accuracy Mean |   Accuracy Std |   Precision Mean |   Precision Std |   Recall Mean |   Recall Std |   F1 Mean |   F1 Std |   ROC Mean |   ROC Std |
|------------+-----------------+----------------+------------------+-----------------+---------------+--------------+-----------+----------+------------+-----------|
| GaussianNB |          0.7295 |        0.00336 |           0.7244 |        0.004383 |        0.9636 |     0.002843 |    0.8271 | 0.002396 |     0.6074 |  0.002692 |
+------------+-----------------+----------------+------------------+-----------------+---------------+--------------+-----------+----------+------------+-----------+
+------------------------+-----------------+----------------+------------------+-----------------+---------------+--------------+-----------+----------+------------+-----